# Convert .reg files to hipscat

## Import libraries

In [17]:
from multiprocessing import cpu_count
from pathlib import Path

import cdshealpix
import dask.distributed
import hipscat.io.file_io
import numpy as np
import lsdb
import mom_builder
import pyarrow as pa
from astropy.coordinates import Angle, Latitude, Longitude
from astropy.table import Table
from hipscat.pixel_math.hipscat_id import HIPSCAT_ID_COLUMN
from hipscat_import.catalog.arguments import ImportArguments as HipscatImportArguments
from hipscat_import.pipeline import pipeline as hipscat_import_pipeline
from mocpy import MOC
from tqdm import tqdm

## Constants

In [18]:
HIGHEST_ORDER = 14  # Smallest tiles to start with
SPLIT_ORDER = 5  # Split building tasks into tiles of this order
DEPTH_DELTA = 2  # Specifies the precision of the healpix filtering, higher is better, lower is faster
MOCPY_THREADS = None  # Number of threads to use with mocpy
MOM_BUILDER_THREADS = cpu_count()  # Number of threads to use with mom_builder
DASK_WORKERS = cpu_count()  # Number of dask workers, for hipscat-import

In [19]:
VALUE_NP_DTYPE = np.dtype('u1')  # Practically the same as np.uint8, but dtype object is required by mom_builder
VALUE_PA_TYPE = pa.from_numpy_dtype(VALUE_NP_DTYPE)
VALUE_COLUMN = 'value'  # Name of the value column in the output Hipscat

In [20]:
REGION_DIR = Path('.')
BOX_REGIONS = REGION_DIR / 'box.reg.I'
DIP_REGIONS = REGION_DIR / 'dip.reg.I.nodups_head'
STARS_REGIONS = REGION_DIR / 'stars.reg.I.nodups_head'

POINTS_MAP_FITS = Path('./hsc-pdr3-forced-r_points_map.fits')

MOM_DIR = Path('./mom_parquet')
HIPSCAT_DIR = Path('./hipscat')
HIPSCAT_NAME = 'hsc_pdr3_mask_i'  # bright star masks for HSC PDR3 I-band

## Read ascii region files and convert to healpix tiles of `HIGHEST_ORDER`

In [21]:
def read_circles_cdshealpix(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra', 'dec', 'radius'],
        units=['deg'] * 3,
    )
    ra = Longitude(table['ra'])
    dec = Latitude(table['dec'])
    radius = Angle(table['radius'])
    
    hp_index = []
    for lon, lat, r in zip(ra, dec, radius):
        idx, _hp_depth, _fully_covered = cdshealpix.cone_search(
            lon,
            lat,
            r,
            depth=HIGHEST_ORDER,
            depth_delta=DEPTH_DELTA,
            flat=True
        )
        hp_index.extend(idx)
        
    return np.array(hp_index, dtype=np.uint64)


def read_circles_mocpy(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra', 'dec', 'radius'],
        units=['deg'] * 3,
    )
    ra = Longitude(table['ra'])
    dec = Latitude(table['dec'])
    radius = Angle(table['radius'])
    
    mocs = MOC.from_cones(
        lon=ra,
        lat=dec,
        radius=radius,
        max_depth=HIGHEST_ORDER,
        delta_depth=DEPTH_DELTA,
        n_threads=MOCPY_THREADS,
    )
    hp_index = np.concatenate([moc.flatten() for moc in mocs])
    return hp_index


# mocpy is faster, even for a single thread
read_circles = read_circles_mocpy


# We use mocpy here because cdshealpix doesn't support box searches
def read_boxes(filename):
    table = Table.read(
        filename,
        format='ascii',
        names=['ra_c', 'dec_c', 'width', 'height'],
        units=['deg'] * 4
    )
    ra_center = Longitude(table['ra_c'])
    dec_center = Latitude(table['dec_c'])
    width = Angle(table['width'])
    height = Angle(table['height'])
    
    width_larger = width > height
    a = np.where(width_larger, 0.5 * width, 0.5 * height)
    b = np.where(width_larger, 0.5 * height, 0.5 * width)
    angle = np.where(width_larger, Angle(90, 'deg'), 0)
    
    mocs = MOC.from_boxes(
        lon=ra_center,
        lat=dec_center,
        a=a,
        b=b,
        angle=angle,
        max_depth=HIGHEST_ORDER,
        n_threads=MOCPY_THREADS,
    )
    hp_index = np.concatenate([moc.flatten() for moc in mocs])
    return hp_index

In [22]:
%time dip_index = read_circles(DIP_REGIONS)
%time stars_index = read_circles(STARS_REGIONS)
%time box = read_boxes(BOX_REGIONS)

CPU times: user 3.72 s, sys: 75.3 ms, total: 3.8 s
Wall time: 1.45 s
CPU times: user 1.63 s, sys: 29.7 ms, total: 1.66 s
Wall time: 1.06 s
CPU times: user 11 s, sys: 142 ms, total: 11.2 s
Wall time: 2.63 s


### Group masks by pixels of `SPLIT_ORDER`

In [23]:
def parent(index, child_order, parent_order):
    delta_order = np.array(child_order - parent_order, dtype=np.uint64)
    assert np.all(delta_order >= 0), 'parent_order must not be lower than the child_order'
    # For some numpyish reason, we cannot just do >>
    shift_bits = np.array(2, dtype=np.uint64) * delta_order
    return np.right_shift(np.array(index, dtype=np.uint64), shift_bits)


def first_child(index, parent_order, child_order):
    delta_order = np.array(child_order - parent_order, dtype=np.uint64)
    assert np.all(delta_order >= 0), 'parent_order must not be lower than the child_order'
    # For some numpyish reason, we cannot just do <<
    shift_bits = np.array(2, dtype=np.uint64) * delta_order
    return np.left_shift(np.array(index, dtype=np.uint64), shift_bits)


def group_by_order(hp_index, hp_order, split_order):
    # Sort and get unique indexes
    hp_index = np.unique(hp_index)
    
    split_order_npix = 12 * (4 ** split_order)
    split_order_indexes = np.arange(1, split_order_npix, dtype=np.uint64)
    hp_order_separation_indexes = first_child(split_order_indexes, split_order, hp_order)
    
    # Split input indexes by the location in split_order tiles
    split_tile_boundaries = np.searchsorted(hp_index, hp_order_separation_indexes)
    lookup = np.split(hp_index,split_tile_boundaries)
    
    return lookup

In [24]:
%%time

dip_lookup = group_by_order(dip_index, HIGHEST_ORDER, SPLIT_ORDER)
stars_lookup = group_by_order(stars_index, HIGHEST_ORDER, SPLIT_ORDER)
box_lookup = group_by_order(box, HIGHEST_ORDER, SPLIT_ORDER)

CPU times: user 546 ms, sys: 32.9 ms, total: 579 ms
Wall time: 586 ms


### Read coverage map (Hipscat's `points_map.fits`) into MOC

In [25]:
coverage = hipscat.io.file_io.read_fits_image(POINTS_MAP_FITS)
# Coverage are bool, but we convert it to the target value dtype
coverage = np.asarray(coverage, dtype=VALUE_NP_DTYPE)

# next I bascially copy HealpixDataset._read_moc_from_point_map()
# https://github.com/astronomy-commons/hipscat/blob/04596506011ec5f021ebe612dcf7f5e8e9044795/src/hipscat/catalog/healpix_dataset/healpix_dataset.py#L114

# length of the file is 12 * 4**order
coverage_order_float = 0.5 * np.log2(coverage.shape[0] / 12)
coverage_order = int(coverage_order_float)
assert coverage_order == coverage_order_float, 'coverage_order must be an integer'
assert coverage_order <= HIGHEST_ORDER, 'coverage_order is larger than HIGHEST_ORDER, cast coverage map to HIGHEST_ORDER'

## Build Multi-Order Continious Map (MOM)

### Define a function to get `HIGHEST_ORDER` tile values

In [26]:
def get_value(order, indexes):
    assert order == HIGHEST_ORDER
    
    first_index = indexes[0]
    top_index = parent(first_index, HIGHEST_ORDER, SPLIT_ORDER)
    indexes_coverage_order = parent(first_index, HIGHEST_ORDER, coverage_order)

    # Get HIGHEST_ORDER indexes and shift them to start from 0
    dip = dip_lookup[top_index] - first_index
    stars = stars_lookup[top_index] - first_index
    box = box_lookup[top_index] - first_index
    
    values = np.zeros(indexes.shape, dtype=VALUE_NP_DTYPE)
    
    # coverage is the first bit, coverage array is either 0 or 1
    values[:] |= coverage[indexes_coverage_order]
    # Masks arrays can be safely shifted by the smallest index value
    # dip is the second bit
    values[dip] |= 1 << 2
    # stars and their boxes is the third bit
    values[stars] |= 1 << 3
    values[box] |= 1 << 3
    
    return values

### Define a MOM merging strategy

In [27]:
mom_merger = mom_builder.MOMMerger(
    # Each healpix tree leaf would have a single value
    # It is a coincidence that we have a column named 'value' 
    state='value',
    # If four children have the same value, they are merged into a single parent
    merger='equal',
    dtype=VALUE_NP_DTYPE,
)

### Initialize parquet writer object

In [28]:
from writer import Writer

writer = Writer(
    MOM_DIR,
    col_name=VALUE_COLUMN,
    col_type=VALUE_PA_TYPE,
)

### Run MOM building pipeline

In [29]:
%%time

generator = mom_builder.gen_mom_from_fn(
    fn=get_value,
    max_norder=HIGHEST_ORDER,
    split_norder=SPLIT_ORDER,
    merger=mom_merger,
    n_threads=MOM_BUILDER_THREADS,
    tiles_consumer=writer.write,  # imimmediately write tiles to parquet, output nothing from the generator
)

# Generator yields nothing, because tiles_consumer is used
for _ in tqdm(generator):
    pass

12289it [00:11, 1090.21it/s]

CPU times: user 1min 23s, sys: 6.98 s, total: 1min 30s
Wall time: 11.3 s


## Create Hipscat with `hipscat-import`

In [30]:
%%time

hipscat_import_args = HipscatImportArguments(
    input_path=MOM_DIR,
    file_reader='parquet',
    output_path=HIPSCAT_DIR,
    output_artifact_name=HIPSCAT_NAME,
    # We have _hipscat_index predefined, it points to the "begging" of a tile
    # (smallest index of order 19 children)
    use_hipscat_index=True,
    add_hipscat_index=False,
) 

with dask.distributed.Client(n_workers=DASK_WORKERS) as client:
    display(client)
    hipscat_import_pipeline(hipscat_import_args)

Planning  :   0%|          | 0/5 [00:00<?, ?it/s]

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65516,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:49159,Total threads: 1
Dashboard: http://127.0.0.1:49161/status,Memory: 2.67 GiB
Nanny: tcp://127.0.0.1:65519,


/Users/hombit/.virtualenvs/elzorro/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49197 instead
  warnings.warn(


Mapping   :   0%|          | 0/12289 [00:00<?, ?it/s]

Binning   :   0%|          | 0/2 [00:00<?, ?it/s]

Splitting :   0%|          | 0/12289 [00:00<?, ?it/s]

Reducing  :   0%|          | 0/36 [00:00<?, ?it/s]

Finishing :   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 38.1 s, sys: 11.6 s, total: 49.7 s
Wall time: 2min 26s


### Load catalog with LSDB

In [32]:
catalog = lsdb.read_hipscat(HIPSCAT_DIR / HIPSCAT_NAME)
with dask.distributed.Client(n_workers=DASK_WORKERS) as client:
    display(client)
    df = catalog.query("value == 13").head(100)
display(df)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49325,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:49352,Total threads: 1
Dashboard: http://127.0.0.1:49353/status,Memory: 2.67 GiB
Nanny: tcp://127.0.0.1:49328,


,pixel_Norder,pixel_Npix,value,Norder,Dir,Npix
_hipscat_index,,,,,,
1161988038834782208,14,270546423,12,0,0,1
1161988064604585984,14,270546429,12,0,0,1
1161988073194520576,14,270546431,12,0,0,1
1161989035267194880,14,270546655,12,0,0,1
1161989125461508096,13,67636669,12,0,0,1
...,...,...,...,...,...,...
1161994979501932544,14,270548039,12,0,0,1
1161994983796899840,13,67637010,12,0,0,1
1161995000976769024,13,67637011,12,0,0,1
